# Toronto Neighborhoods clustering

Firstly we generate the dataframe as shown in the previous section.

## section 1

In [1]:
#importing necessary libraries
from bs4 import BeautifulSoup
import lxml
import requests
import pandas as pd

In [2]:
#storing the html code of table as text from the link
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')
table=soup.find('table')

In [3]:
#getting column headers
columns=[]
for column_name in table.find_all('th'):
    columns.append(column_name.text.split('\n')[0])
df=pd.DataFrame(columns=columns)

In [4]:
#getting the data from the rows of table
column1=[]
column2=[]
column3=[]

for data in table.tbody.find_all('tr'):
    try:
        column1.append(data.find_all('td')[0].text.split('\n')[0])
        column2.append(data.find_all('td')[1].text.split('\n')[0])
        column3.append(data.find_all('td')[2].text.split('\n')[0])
    except:
        pass
df['Postal code']=column1
df['Borough']=column2
df['Neighborhood']=column3

In [5]:
#cleaning table by removing empty data
df=df[df['Borough']!='Not assigned'].reset_index().drop('index',axis=1)

In [6]:
#replacing '/' with ',' in neighborhood
column3=[]
for data in df['Neighborhood']:
    column3.append(','.join(map(str,data.split('/'))))
df['Neighborhood']=column3   

In [7]:
df.head()

Postal code           Borough                                  Neighborhood
0         M3A        North York                                     Parkwoods
1         M4A        North York                              Victoria Village
2         M5A  Downtown Toronto                    Regent Park , Harbourfront
3         M6A        North York             Lawrence Manor , Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park , Ontario Provincial Government

In [8]:
df.shape

(103, 3)

## section 2

Now fetching the postal code latitude and longitude by the csv file given,

In [9]:
# reading the csv for postal code information
postal=pd.read_csv('http://cocl.us/Geospatial_data')
postal.rename(columns={'Postal Code':'Postal code'},inplace=True)

In [10]:
#Adding latitude and longitude in df
df=df.sort_values('Postal code').reset_index().drop('index',1)
df=df.merge(postal)

In [11]:
df.head()

Postal code      Borough                              Neighborhood  \
0         M1B  Scarborough                           Malvern , Rouge   
1         M1C  Scarborough  Rouge Hill , Port Union , Highland Creek   
2         M1E  Scarborough       Guildwood , Morningside , West Hill   
3         M1G  Scarborough                                    Woburn   
4         M1H  Scarborough                                 Cedarbrae   

    Latitude  Longitude  
0  43.806686 -79.194353  
1  43.784535 -79.160497  
2  43.763573 -79.188711  
3  43.770992 -79.216917  
4  43.773136 -79.239476